# Tools

In [84]:
# basic
import datetime as dt
import pandas as pd

# crawler
import requests
from bs4 import BeautifulSoup

# gsheet
import pygsheets
import modules.settings as s

# settings
pd.options.display.max_columns=None
pd.set_option('max_colwidth', None)  # 設定 value 顯示長度

# modules
> **爬取資訊**
    >- 發文日期
    >- 看板
    >- 文章標題
    >- 推文數    
    >- 作者
    >- 文章連結
    
> **Functions**
>- **get_webPage (url)**
    - 功能：取得網頁原始碼。
    - Return：網頁原始碼。
    - 參數
        - url：欲爬取的 PTT 看板網址。
>- **get_singlePage_posts (page)**
    - 功能：取得單一頁面所有推文。
    - Return：單一頁面所有推文（dict 格式）、上一頁 url。
    - 參數
        - page：get_webPage (url) return 的資料。
>- **get_ptt_posts (url, return_pages)**
    - 功能：爬取 PTT 指定看板推文。
    - Return：DataFrame 格式資料。
    - 參數
        - url：欲爬取的 PTT 看板網址。
        - return_pages：欲往回爬取幾頁的資料。（default = 0，表示僅爬取最新頁面資料）

In [76]:
def get_webPage(url):
    r = requests.get(url=url, cookies={'over18': '1'})  # cookies：有些看板會擋是否已滿18歲
    if r.status_code != 200:
        print('URL 無效，status code = ', r.status_code)
    else:
        return r.text
    
def get_singlePage_posts(page):
    # parse
    soup = BeautifulSoup(page, "html.parser")
    divs = soup.find_all('div', class_='r-ent')
    
    # 頁面資訊
    PTT_URL = "https://www.ptt.cc"
    board = soup.find('a', 'board').text.lstrip('看板 ')  # 看板
    
    page_div = soup.find('div', 'btn-group btn-group-paging')  # 控制頁面區塊
    prev_page_url = PTT_URL + page_div.find_all('a')[1]['href']  # 上頁
#     latest_page_url = PTT_URL + page_div.find_all('a')[3]['href']  # 最新
    
    # 日期
    today = dt.datetime.today().date().strftime('%m/%d').lstrip('0')
    
    # 需求欄位
    dates=[]
    titles=[]
    push_counts=[]
    authors=[]
    hrefs=[]
    
    # 最新頁面
    for article in divs:
        date = article.find('div', 'date').text.strip()  # strip(): 去除字串頭尾的空格

        # 排除已刪除貼文
        if article.find('div', 'title').a == None:
            continue
        else:
            # 需求資訊
            title = article.find('div', 'title').a.text  # 標題
            push_count = article.find('div', 'nrec').text  # 推文數 (str)
            author = article.find('div', 'author').text  # author
            href = PTT_URL + article.find('a')['href']  # URL = https://www.ptt.cc + href

            dates.append(date)
            titles.append(title)
            push_counts.append(push_count)
            authors.append(author)
            hrefs.append(href)
    
    # return 最新頁面的推文
    posts = {'date': dates, 
            'board': board, 
            'title': titles, 
            'push_count': push_counts, 
            'author': authors, 
            'href': hrefs}
    
    return posts, prev_page_url

def get_ptt_posts(url, return_pages=0):
    # 先爬取最新頁面的所有推文
    all_posts = []
    page = get_webPage(url)
    posts, prev_page_url = get_singlePage_posts(page)
    all_posts.append(posts)

    # 再往前 return_pages 頁
    count = 0
    while count < return_pages:
        page = get_webPage(prev_page_url)
        posts, prev_page_url = get_singlePage_posts(page)
        all_posts.append(posts)
        count += 1
    
    # 轉換為 DataFrame
    df = pd.DataFrame()
    for post in all_posts:
        df = pd.concat([df, pd.DataFrame(post)])
        
    df.reset_index(inplace=True, drop=True)
    
    return df

# 爬取特定看板資訊

## NBA

In [78]:
url = 'https://www.ptt.cc/bbs/NBA/index.html'
nba = get_ptt_posts(url, 3)
nba

,date,board,title,push_count,author,href
0,2/09,NBA,[情報] 紐約疫苗規定或將取消,56,FengQQ,https://www.ptt.cc/bbs/NBA/M.1644378138.A.1DE.html
1,2/09,NBA,[花邊] NAW和Josh Hart被交易後坐在場邊看球,17,willy911006,https://www.ptt.cc/bbs/NBA/M.1644378329.A.AD6.html
2,2/09,NBA,[BOX ] Pistons 86:116 Mavericks 數據,21,Rambo,https://www.ptt.cc/bbs/NBA/M.1644378557.A.75C.html
3,2/09,NBA,Fw: [情報] 高雄鋼鐵人新洋將 Bennett,爆,asdfgh0920,https://www.ptt.cc/bbs/NBA/M.1644379310.A.39B.html
4,11/22,NBA,[公告] 板規v9.0,1,pneumo,https://www.ptt.cc/bbs/NBA/M.1637555315.A.C34.html
...,...,...,...,...,...,...
57,2/08,NBA,Re: [討論] 西河其實蠻冤枉的,,AOB123,https://www.ptt.cc/bbs/NBA/M.1644326621.A.4CE.html
58,2/08,NBA,Re: [討論] 西河其實蠻冤枉的,25,bangch,https://www.ptt.cc/bbs/NBA/M.1644327291.A.C2D.html
59,2/08,NBA,[花邊] DJM 七年前的推特說總有一天要進明星賽,64,Petyr,https://www.ptt.cc/bbs/NBA/M.1644328051.A.F2C.html
60,2/08,NBA,[情報] 湖人隊向塞爾提克洽談 Josh Richardson,51,thnlkj0665,https://www.ptt.cc/bbs/NBA/M.1644330386.A.8B6.html


## 八卦版

In [79]:
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
gossiping = get_ptt_posts(url, 3)
gossiping

,date,board,title,push_count,author,href
0,2/09,Gossiping,[協尋] 台中貴賓狗協尋賞金60萬,,jhct110,https://www.ptt.cc/bbs/Gossiping/M.1644380284.A.D94.html
1,2/09,Gossiping,Re: [問卦] 買15元麥香的是不是傻子？,,chen0625,https://www.ptt.cc/bbs/Gossiping/M.1644380292.A.214.html
2,2/09,Gossiping,[問卦] 你各位電視現在是Mod還是第四台？,,w1230319,https://www.ptt.cc/bbs/Gossiping/M.1644380323.A.DEC.html
3,2/09,Gossiping,[問卦] 讓外送員等多久是合理的？,,acln0816,https://www.ptt.cc/bbs/Gossiping/M.1644380335.A.C2D.html
4,2/09,Gossiping,Re: [新聞] 高雄民族路機車速限40 引鄉民論戰,,A6,https://www.ptt.cc/bbs/Gossiping/M.1644380344.A.8CA.html
...,...,...,...,...,...,...
64,2/09,Gossiping,[問卦] 大樓蓋一堆公設要幹嘛又用不到,10,cj4385283,https://www.ptt.cc/bbs/Gossiping/M.1644379114.A.541.html
65,2/09,Gossiping,[新聞] 陳時中鬆口不排斥參選 蔣萬安：現在不適,12,runa2,https://www.ptt.cc/bbs/Gossiping/M.1644379154.A.8F9.html
66,2/09,Gossiping,[問卦] 核四弊案為什麼沒事?,,cattgirl,https://www.ptt.cc/bbs/Gossiping/M.1644379170.A.5DE.html
67,2/09,Gossiping,[爆卦] 李玟誼冬奧高山滑雪晉級決賽!,21,Borges,https://www.ptt.cc/bbs/Gossiping/M.1644379226.A.BD5.html


## Stock

In [80]:
url = 'https://www.ptt.cc/bbs/Stock/index.html'
stock = get_ptt_posts(url, 3)
stock

,date,board,title,push_count,author,href
0,2/09,Stock,[新聞] 世貿組織認定美對韓加徵洗衣機進口關稅,31,CavendishJr,https://www.ptt.cc/bbs/Stock/M.1644365672.A.1A5.html
1,2/09,Stock,Re: [新聞] 美管制陸企 砍向面板業,19,winall,https://www.ptt.cc/bbs/Stock/M.1644365818.A.99C.html
2,2/09,Stock,[閒聊] 2022/02/09盤中閒聊,爆,justforsing,https://www.ptt.cc/bbs/Stock/M.1644366602.A.865.html
3,2/09,Stock,[新聞] 產業分析》大摩：IT面板帶來不確定性 中,32,rubberking,https://www.ptt.cc/bbs/Stock/M.1644370796.A.AA4.html
4,2/09,Stock,[新聞] 歡迎台積電合作！ 歐盟斥資1.37兆堆晶片,57,s3z15a3z15a,https://www.ptt.cc/bbs/Stock/M.1644375753.A.500.html
...,...,...,...,...,...,...
61,2/08,Stock,[情報] 富旺處分陽明普通股股票之公告,25,village0817,https://www.ptt.cc/bbs/Stock/M.1644313553.A.82A.html
62,2/08,Stock,[情報] 6412群電 1月營收,5,nodnarb1027,https://www.ptt.cc/bbs/Stock/M.1644313904.A.638.html
63,2/08,Stock,[情報] 4958 臻鼎-KY 1月營收,26,nodnarb1027,https://www.ptt.cc/bbs/Stock/M.1644314409.A.F77.html
64,2/08,Stock,Re: [新聞] 瑞幸咖啡：已繳納1.8億美元罰款,8,danny7373,https://www.ptt.cc/bbs/Stock/M.1644314633.A.6F5.html


## money

In [82]:
url = 'https://www.ptt.cc/bbs/money/index.html'
money = get_ptt_posts(url, 3)
money

,date,board,title,push_count,author,href
0,6/23,money,Fw: [心得] 2609陽明目標價之我見-20年歷史數據分析,5,az61007,https://www.ptt.cc/bbs/money/M.1624450998.A.24E.html
1,7/15,money,[新聞] 女股神：比特幣環保擔憂只是一時 長期仍,,hateOnas,https://www.ptt.cc/bbs/money/M.1626329131.A.EF3.html
2,7/26,money,Fw: [請益] 從工作經驗推論航運業,,hateOnas,https://www.ptt.cc/bbs/money/M.1627267068.A.D4A.html
3,8/02,money,[其他] money and currency,,hsiliang,https://www.ptt.cc/bbs/money/M.1627918377.A.8C1.html
4,8/03,money,Fw: [公告] 8/8 停機公告,2,az61007,https://www.ptt.cc/bbs/money/M.1627982048.A.856.html
...,...,...,...,...,...,...
72,4/11,money,Re: [問題] 想知道特地過來酸的心態是？,15,crazybaby,https://www.ptt.cc/bbs/money/M.1618154927.A.3CC.html
73,4/12,money,[公告] drgon違反群組規十一依臨時板規水桶30天,45,IanLi,https://www.ptt.cc/bbs/money/M.1618179846.A.90F.html
74,4/12,money,Re: [建議] 趕快離開這個板來CFP,6,walter741225,https://www.ptt.cc/bbs/money/M.1618182368.A.AE7.html
75,4/12,money,Re: [公告] drgon違反群組規十一依臨時板規水桶30天,,st90513,https://www.ptt.cc/bbs/money/M.1618184502.A.542.html


In [83]:
money.query('date == "2/09"')

,date,board,title,push_count,author,href
14,2/09,money,[標的] 網龍 3083 短多,42,drgon,https://www.ptt.cc/bbs/money/M.1644370114.A.54E.html


# 串接 Google sheet

> Ref
>- [[Python爬蟲教學]解析如何串接Google Sheet試算表寫入爬取的資料](https://www.learncodewithmike.com/2020/08/python-write-to-google-sheet.html)
>- [Python 讀寫 Google Sheets 教學](https://hackmd.io/@Yun-Cheng/GoogleSheets)

## gsheet 認證

In [85]:
gc = pygsheets.authorize(service_account_file=s.GSHEETS_PYTHON_CONNECT_PATH)

# Google sheet URL
survey_url = 'https://docs.google.com/spreadsheets/d/1kI3AECFBEBFbOo4QxL1SCyA8ekcxROLkkO_5ocXQatU/edit#gid=0'

sh = gc.open_by_url(survey_url)

## 讀取目標工作表

In [90]:
ws_NBA = sh.worksheet_by_title('NBA')

In [91]:
ws_Gossiping = sh.worksheet_by_title('Gossiping')

## 寫入工作表
- [set_dataframe()](https://pygsheets.readthedocs.io/en/stable/worksheet.html#pygsheets.Worksheet.set_dataframe)

In [94]:
ws_NBA.set_dataframe(nba, start='B2')

In [95]:
ws_Gossiping.set_dataframe(gossiping, start='B2')

## 讀取目標工作表後，將工作表轉換為 DataFrame
- [set_as_df()](https://pygsheets.readthedocs.io/en/stable/worksheet.html#pygsheets.Worksheet.get_as_df)

In [96]:
nba_gsheet = ws_NBA.get_as_df(start='B2')
nba_gsheet

,date,board,title,push_count,author,href
0,02/09,NBA,[情報] 紐約疫苗規定或將取消,56,FengQQ,https://www.ptt.cc/bbs/NBA/M.1644378138.A.1DE.html
1,02/09,NBA,[花邊] NAW和Josh Hart被交易後坐在場邊看球,17,willy911006,https://www.ptt.cc/bbs/NBA/M.1644378329.A.AD6.html
2,02/09,NBA,[BOX ] Pistons 86:116 Mavericks 數據,21,Rambo,https://www.ptt.cc/bbs/NBA/M.1644378557.A.75C.html
3,02/09,NBA,Fw: [情報] 高雄鋼鐵人新洋將 Bennett,爆,asdfgh0920,https://www.ptt.cc/bbs/NBA/M.1644379310.A.39B.html
4,11/22,NBA,[公告] 板規v9.0,1,pneumo,https://www.ptt.cc/bbs/NBA/M.1637555315.A.C34.html
...,...,...,...,...,...,...
57,02/08,NBA,Re: [討論] 西河其實蠻冤枉的,,AOB123,https://www.ptt.cc/bbs/NBA/M.1644326621.A.4CE.html
58,02/08,NBA,Re: [討論] 西河其實蠻冤枉的,25,bangch,https://www.ptt.cc/bbs/NBA/M.1644327291.A.C2D.html
59,02/08,NBA,[花邊] DJM 七年前的推特說總有一天要進明星賽,64,Petyr,https://www.ptt.cc/bbs/NBA/M.1644328051.A.F2C.html
60,02/08,NBA,[情報] 湖人隊向塞爾提克洽談 Josh Richardson,51,thnlkj0665,https://www.ptt.cc/bbs/NBA/M.1644330386.A.8B6.html


In [97]:
gossiping_gsheet = ws_Gossiping.get_as_df(start='B2')
gossiping_gsheet

,date,board,title,push_count,author,href
0,02/09,Gossiping,[協尋] 台中貴賓狗協尋賞金60萬,,jhct110,https://www.ptt.cc/bbs/Gossiping/M.1644380284.A.D94.html
1,02/09,Gossiping,Re: [問卦] 買15元麥香的是不是傻子？,,chen0625,https://www.ptt.cc/bbs/Gossiping/M.1644380292.A.214.html
2,02/09,Gossiping,[問卦] 你各位電視現在是Mod還是第四台？,,w1230319,https://www.ptt.cc/bbs/Gossiping/M.1644380323.A.DEC.html
3,02/09,Gossiping,[問卦] 讓外送員等多久是合理的？,,acln0816,https://www.ptt.cc/bbs/Gossiping/M.1644380335.A.C2D.html
4,02/09,Gossiping,Re: [新聞] 高雄民族路機車速限40 引鄉民論戰,,A6,https://www.ptt.cc/bbs/Gossiping/M.1644380344.A.8CA.html
...,...,...,...,...,...,...
64,02/09,Gossiping,[問卦] 大樓蓋一堆公設要幹嘛又用不到,10,cj4385283,https://www.ptt.cc/bbs/Gossiping/M.1644379114.A.541.html
65,02/09,Gossiping,[新聞] 陳時中鬆口不排斥參選 蔣萬安：現在不適,12,runa2,https://www.ptt.cc/bbs/Gossiping/M.1644379154.A.8F9.html
66,02/09,Gossiping,[問卦] 核四弊案為什麼沒事?,,cattgirl,https://www.ptt.cc/bbs/Gossiping/M.1644379170.A.5DE.html
67,02/09,Gossiping,[爆卦] 李玟誼冬奧高山滑雪晉級決賽!,21,Borges,https://www.ptt.cc/bbs/Gossiping/M.1644379226.A.BD5.html
